In [10]:
max_item_count = 24
rarity_thershold = 80

In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
from collections import defaultdict

In [12]:
csv_file_path_train = './train_dataset.csv'
csv_file_path_test = './submission_data_x.csv'

df_train = pd.read_csv(csv_file_path_train)
df_test = pd.read_csv(csv_file_path_test)

/var/folders/6h/g5v5834n5mgg95k1hkl9_yc40000gn/T/ipykernel_4585/896890968.py:4: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23,24,59,60,61,62,63,64,65,66,67,68,69,70,71,72,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,102,103,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(csv_file_path_train)
/var/folders/6h/g5v5834n5mgg95k1hkl9_yc40000gn/T/ipykernel_4585/896890968.py:5: DtypeWarning: Columns (97,105,107,108,109,110,111,112,113,114,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv(csv_file_path_test)


In [13]:
df_train = pd.read_csv(csv_file_path_train)

/var/folders/6h/g5v5834n5mgg95k1hkl9_yc40000gn/T/ipykernel_4585/222863778.py:1: DtypeWarning: Columns (11,12,13,14,15,16,17,18,19,20,21,22,23,24,59,60,61,62,63,64,65,66,67,68,69,70,71,72,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,102,103,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(csv_file_path_train)


In [14]:
# Manage the TEST data
col_list = ["item", "make", "model", "goods_code"]
all_colmns_to_append = []
for clmn_name in col_list:

    # Extract uniqueness and repetiteveness
    row_count = df_test.shape[0]
    count_dict = defaultdict(int)
    
    cols = {clmn_name+str(i) for i in range(1,max_item_count+1)}

    for col in cols:
        column_values = df_test[col]
        # del df_test[col]
        for row in range(row_count - 1):
            item = column_values[row]
            if pd.notna(item):
                count_dict[str(item)] += 1

    count_dict = sorted(count_dict.items(), key=lambda x: x[1], reverse=True)
    
    # ADD COLUMNS  
    for key, count in count_dict:
        if count > rarity_thershold:
            all_colmns_to_append.append(str(key))

all_colmns_to_append = list(set(all_colmns_to_append))
new_data = pd.DataFrame(0, index=df_test.index, columns=all_colmns_to_append)
df_test = pd.concat([df_test, new_data], axis=1)

for clmn_name in col_list:
    cols = {clmn_name+str(i) for i in range(1,max_item_count+1)}
    # FILL NEWLY ADDED COLUMNS
    for index, row in df_test.iterrows():
        for col in cols:
            cell = row[col]
            if not pd.notna(cell):
                break
            if cell in df_test.columns:
                df_test.at[index, cell] = 1

    # DROP OLD COLUMNS
    for col in cols:
        del df_test[col]

In [15]:
# Manage the TRAIN data

# ADD COLUMNS
cols_df_test = list(df_test.columns)
cols_df_train = list(df_train.columns)
cols_add_train = []

for col in cols_df_test:
    if col not in cols_df_train:
        cols_add_train.append(col)

new_data = pd.DataFrame(0, index=df_train.index, columns=cols_add_train)
df_train = pd.concat([df_train, new_data], axis=1)

for clmn_name in col_list:

    cols = {clmn_name+str(i) for i in range(1,max_item_count+1)}

    # # FILL NEWLY ADDED COLUMNS
    for index, row in df_train.iterrows():
        for col in cols:
            cell = row[col]
            if not pd.notna(cell):
                break
            if cell in df_train.columns:
                df_train.at[index, cell] = 1

    # DROP OLD COLUMNS
    for col in cols:
        del df_train[col]

In [16]:
df_train = df_train.fillna(0)

df_test = df_test.fillna(0)

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

train_y = df_train['fraud_flag']  
train_x = df_train.drop(columns=['fraud_flag'])  


model = RandomForestClassifier()  
model.fit(train_x, train_y)

predictions = model.predict(df_test)
probabilities = model.predict_proba(df_test)[:, 1]  


print("Predictions:", predictions)
print("Probabilities:", probabilities)




Predictions: [0 0 0 ... 0 0 0]
Probabilities: [0.24 0.01 0.   ... 0.   0.   0.  ]


In [21]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


train_y = df_train['fraud_flag']
train_x = df_train.drop(columns=['fraud_flag'])


model = XGBClassifier() 
model.fit(train_x, train_y)


predictions = model.predict(df_test)
probabilities = model.predict_proba(df_test)[:, 1] 

print("Predictions:", predictions)
print("Probabilities:", probabilities)


Predictions: [0 0 0 ... 0 0 0]
Probabilities: [0.00075525 0.00607294 0.00742015 ... 0.00166458 0.00741968 0.00547982]


In [23]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import average_precision_score

df_train = df_train.sample(frac=1).reset_index(drop=True)

train_y = df_train['fraud_flag']
train_x = df_train.drop(columns=['fraud_flag'])

X_train, X_new_test, y_train, y_new_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [24]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_new_test_probabilities = rf_model.predict_proba(X_new_test)[:, 1]

xgb_model = XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
xgb_new_test_probabilities = xgb_model.predict_proba(X_new_test)[:, 1]

results_df = pd.DataFrame({'Actual': y_new_test, 
                           'RF_Predicted_Probability': rf_new_test_probabilities,
                           'XGB_Predicted_Probability': xgb_new_test_probabilities})

results_df.to_csv('new_test_results.txt', index=False, sep='\t')

rf_average_precision = average_precision_score(results_df['Actual'], results_df['RF_Predicted_Probability'])
print(f"Random Forest Average Precision Score: {rf_average_precision}")

xgb_average_precision = average_precision_score(results_df['Actual'], results_df['XGB_Predicted_Probability'])
print(f"XGBoost Average Precision Score: {xgb_average_precision}")


Random Forest Average Precision Score: 0.19270252343162442
XGBoost Average Precision Score: 0.21311064481889538


In [26]:
xgb_new_test_probabilities = xgb_model.predict_proba(df_test)[:, 1]


In [27]:
with open("sample_submission.csv", "r") as file:
    lines = file.readlines()


new_lines = []
for i, prob in enumerate(xgb_new_test_probabilities):
    new_lines.append(f"{lines[i + 1].strip()}{prob}\n") 

with open("results.csv", "w") as file:
    file.write("ID,fraud_flag\n")  
    file.writelines(new_lines)
